# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import sqlite3
import pandas as pd
import nltk
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
#Querying the Disaster table using pandas read_sql method
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_messages',con=engine)

In [22]:
X = df['message'] 
Y = df.iloc[:,4:]

In [23]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [24]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [25]:
def tokenize(text):
    normalized_text = re.sub(r'[^a-zA-Z0-9]',' ',str(text)).lower()
    word_tokens = word_tokenize(normalized_text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    tokens = [lemmatizer.lemmatize(word) for word in word_tokens  if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
#Creating a Pipeline
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer = tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10,min_samples_leaf=1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

In [28]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
y_pred = pipeline.predict(X_test)

In [33]:
y_test.keys()

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [34]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [110]:
y_test.shape

(6554, 36)

In [111]:
y_pred.shape

(6554, 36)

In [35]:
Y.columns.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object)

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,target_names=y_test.keys()))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      5003
               request       0.80      0.44      0.57      1087
                 offer       0.00      0.00      0.00        25
           aid_related       0.74      0.60      0.67      2693
          medical_help       0.52      0.11      0.18       509
      medical_products       0.69      0.08      0.14       309
     search_and_rescue       0.62      0.04      0.07       201
              security       0.00      0.00      0.00       118
              military       0.69      0.11      0.19       222
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.32      0.46       369
                  food       0.80      0.54      0.64       685
               shelter       0.81      0.38      0.52       570
              clothing       0.81      0.19      0.31       115
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
parameters = {
    'clf__estimator__n_estimators': [5, 10],
    }

cv = GridSearchCV(pipeline,param_grid=parameters,cv = 5)

In [38]:
cv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [39]:
y_pred_grid = cv.predict(X_test)

In [40]:
print(classification_report(y_test, y_pred_grid,target_names=y_test.keys()))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      5003
               request       0.81      0.44      0.57      1087
                 offer       0.00      0.00      0.00        25
           aid_related       0.74      0.61      0.67      2693
          medical_help       0.49      0.08      0.14       509
      medical_products       0.69      0.09      0.17       309
     search_and_rescue       0.45      0.02      0.05       201
              security       0.00      0.00      0.00       118
              military       0.64      0.10      0.18       222
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.20      0.31       369
                  food       0.79      0.59      0.68       685
               shelter       0.81      0.27      0.41       570
              clothing       0.84      0.18      0.30       115
                 money       0.62      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [41]:
best_model = cv.best_params_

In [42]:
best_model

{'clf__estimator__n_estimators': 10}

In [10]:
#Lets Create a new pipeline with below hyperparameters
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,min_samples_leaf=10,max_features=0.5,n_jobs=-1)))
    ])

In [11]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [14]:
y_pred_new = pipeline2.predict(X_test)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_new,target_names=y_test.keys()))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5060
               request       0.76      0.50      0.61      1151
                 offer       0.00      0.00      0.00        22
           aid_related       0.71      0.69      0.70      2689
          medical_help       0.56      0.35      0.43       489
      medical_products       0.72      0.30      0.42       321
     search_and_rescue       0.64      0.15      0.25       188
              security       0.00      0.00      0.00       118
              military       0.54      0.37      0.44       209
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.65      0.70       417
                  food       0.80      0.78      0.79       740
               shelter       0.77      0.62      0.69       544
              clothing       0.75      0.51      0.61       100
                 money       0.71      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [43]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(best_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.